# OSNAP line Lagrangian particle tracking investigation of the cold/fresh blob


### The aim here is an investigation of the source of the cold/fresh blob which appeared in the near-surface Iceland Basin from about 2014. There are a couple of published explanations for the feature: 

- reduced heat/salt transport across RAPID array over several years propagating northwards (Bryden et al. 2020?) or 
- increased contribution from Labrador Sea water flowing into subpolar North Atlantic rather than southwards (Holliday et al. 2020?). Driven I think by specific,  more intermittent, weather near Flemish Cap (though I need to read this again...)

The technique is to track particles back from the OSNAP line, initially for 2 years. Each particle has an associated transport normal to OSNAP, which I use to estimate the contribution to volumes/heat/salt crossing OSNAP northwards from the two main routes - Gulf Stream and Labrador Sea. And see how that changes over the years from 2006 to 2017.

The main conclusion is that from about 2012 Labrador Sea becomes a more important source of surface waters to the Iceland Basin, but there is also a reduction in the temperature and salinity of the water from the Gulf Stream. So probably a balance of both. **But I need to run the particles back further - 2 years is not long enough to pinpoint the pathway for most particles.**

## Technical preamble

In [ ]:
# import matplotlib.colors as colors
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
# from matplotlib.colors import ListedColormap
import cmocean as co
import pandas as pd
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import seawater as sw

from matplotlib import colors as c
from matplotlib import ticker
from xhistogram.xarray import histogram

sns.set(style="darkgrid")

xr.set_options(keep_attrs=True)
np.warnings.filterwarnings('ignore')

sns.set_palette("colorblind")
xr.set_options(keep_attrs=True);


### Set up paths and read in Viking20 section data

In [ ]:
# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# Section data file
path_data_section = Path('data/processed/sections/') 
fname0 = f'osnapSection2000to09.nc'
fname1 = f'osnapSection2010to18.nc'


In [ ]:
section0 = xr.open_dataset(project_path / path_data_section / fname0)
section1 = xr.open_dataset(project_path / path_data_section / fname1)
section = xr.concat((section0,section1),dim="time_counter")

# add some missing attributes, mostly for automatic axis labels

section.vosaline.attrs['long_name']='salinity'
section.vosaline.attrs['units']='PSU'
section.votemper.attrs['long_name']='potential temperature'
section.votemper.attrs['units']='$\degree C$'
section.c.attrs['long_name']='Distance'
section.c.attrs['units']='m'
section.z.attrs['long_name']='Depth'
section.z.attrs['units']='m'
section.time_counter.attrs["long_name"] = 'Time'

# concat introduces an extra time_counter axis on the variables which didn't have one originally
# so get rid of it

section['ii'] = section['ii'].isel(time_counter=0).drop('time_counter')
section['jj'] = section['jj'].isel(time_counter=0).drop('time_counter')
section['dx'] = section['dx'].isel(time_counter=0).drop('time_counter')
section['lat'] = section['lat'].isel(time_counter=0).drop('time_counter')
section['lon'] = section['lon'].isel(time_counter=0).drop('time_counter')
section['dz'] = section['dz'].isel(time_counter=0).drop('time_counter')
section['mask'] = section['mask'].isel(time_counter=0).drop('time_counter')


In [ ]:
def plot_hoffmuller():
    fig,ax = plt.subplots(1,2,figsize=(15,13),sharey=True,gridspec_kw={'wspace': 0.1})
    plt.tight_layout()
    section.votemper.isel(z=15).plot(ax=ax[0],vmin=0,cmap=co.cm.thermal,cbar_kwargs={'orientation':'horizontal','pad':0.1})
    section.vosaline.isel(z=15).plot(ax=ax[1],vmin=34.6,cmap=co.cm.haline,cbar_kwargs={'orientation':'horizontal','pad':0.1})

    ax[1].set_ylabel('')

    ax[0].ticklabel_format(axis='x', style='sci', scilimits=(-3,3))
    ax[1].ticklabel_format(axis='x', style='sci', scilimits=(-3,3))

    # # Hide x labels and tick labels for top plots and y ticks for right plots.
    # for axis in ax.flat:
    #     axis.label_outer()
    return

##  Hoffmuller diagram of temperature and salinity on the OSNAP section from Viking20x

In [ ]:
plot_hoffmuller()

In [ ]:
def plot_ts_timeseries():
    fig,ax = plt.subplots(2,figsize=(15,10),sharex=True,gridspec_kw={'wspace': 0.1})
    plt.tight_layout()

    section.votemper.isel(z=15).isel(c=635).plot(ax=ax[0],label='Irminger Basin 37.5$\degree$W')
    section.votemper.isel(z=15).isel(c=742).plot(ax=ax[0],label='Irminger Basin 32.6$\degree$W')
    section.votemper.isel(z=15).isel(c=835).plot(ax=ax[0],label='Iceland Basin 28.9$\degree$W')
    section.votemper.isel(z=15).isel(c=970).plot(ax=ax[0],label='Iceland Basin 22.0$\degree$W')
    section.votemper.isel(z=15).isel(c=1050).plot(ax=ax[0],label='Hatton-Rockall Bank 17.6$\degree$W')
    section.votemper.isel(z=15).isel(c=1200).plot(ax=ax[0],label='Rockall Trough 9.8$\degree$W')
    
    ax[0].axvline(np.datetime64('2006-01-16T12:00:00'))
    ax[0].axvline(np.datetime64('2017-07-16T12:00:00'))

    section.vosaline.isel(z=15).isel(c=635).plot(ax=ax[1],label='Irminger Basin 37.5$\degree$W')
    section.vosaline.isel(z=15).isel(c=742).plot(ax=ax[1],label='Irminger Basin 32.6$\degree$W')
    section.vosaline.isel(z=15).isel(c=835).plot(ax=ax[1],label='Iceland Basin 28.9$\degree$W')
    section.vosaline.isel(z=15).isel(c=970).plot(ax=ax[1],label='Iceland Basin 22.0$\degree$W')
    section.vosaline.isel(z=15).isel(c=1050).plot(ax=ax[1],label='Hatton-Rockall Bank 17.6$\degree$W')
    section.vosaline.isel(z=15).isel(c=1200).plot(ax=ax[1],label='Rockall Trough 9.8$\degree$W')

    ax[1].axvline(np.datetime64('2006-01-16T12:00:00'))
    ax[1].axvline(np.datetime64('2017-07-16T12:00:00'))

    ax[0].set_xlabel('')

    ax[0].legend()
    ax[1].legend()
    return

## Timeseries of temperature and salinity at points on the OSNAP section. Vertical lines show the earliest and latest particle releases.

In [ ]:
plot_ts_timeseries()

## Set up paths and read in trajectory data

In [ ]:
## Parameters
# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# Parcels track data file
path_data_tracks = Path('data/processed/tracks/osnap/') 
filename_tracks = ['tracks_osnap_backward_200607_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_200707_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_200807_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_200907_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201007_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201107_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201207_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201307_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201407_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201507_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201607_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201707_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_200601_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_200701_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_200801_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_200901_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201001_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201101_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201201_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201301_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201401_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201501_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201601_N19886_D730_Rnd123.nc',
                   'tracks_osnap_backward_201701_N19886_D730_Rnd123.nc']

# model mask file
data_path = Path("data/external/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = project_path / data_path / "mask" / experiment_name / "1_mesh_mask.nc"

#section lonlat file
sectionPath = Path('data/external/sections/')
sectionFilename = 'osnap_pos_wp.txt'
sectionname = 'osnap'
gsrsectionFilename = 'gsr_pos_wp.txt'

degree2km = 1.852*60.0


# some transport values specific to osnap runs
# randomly seeded 39995 particles, 19886 were in ocean points (the rest were land)

osnap_section_length = 3594572.87839    # m
osnap_section_depth = 4000 # m over which particles launched

osnap_section_ocean_area = osnap_section_length * osnap_section_depth * 19886.0 / 39995.0

particle_section_area = osnap_section_length * osnap_section_depth / 39995.0


## Load data

### mesh and masks

In [ ]:
mesh_mask = xr.open_dataset(mesh_mask_file)
mesh_mask = mesh_mask.squeeze()
mesh_mask = mesh_mask.set_coords(["nav_lon", "nav_lat", "nav_lev"])

bathy = mesh_mask.mbathy.rename("number of water filled points")

depth = (mesh_mask.e3t_0 * mesh_mask.tmask).sum("z")
# display(mesh_mask)

### section position data

In [ ]:
lonlat = xr.Dataset(pd.read_csv(project_path / sectionPath / sectionFilename,delim_whitespace=True))
lonlat['lon'] *= -1.0

In [ ]:
lonlat.lon.attrs['long_name']='Longitude'
lonlat.lat.attrs['long_name']='Latitude'
lonlat.lon.attrs['standard_name']='longitude'
lonlat.lat.attrs['standard_name']='latitude'
lonlat.lon.attrs['units']='degrees_east'
lonlat.lat.attrs['units']='degrees_north'

lonlat2mean= lonlat.rolling({'dim_0':2}).mean()

lonlatdiff = (lonlat.diff('dim_0'))

lonlatdiff = lonlatdiff.assign({'y':lonlatdiff['lat']*degree2km})
lonlatdiff = lonlatdiff.assign({'x':lonlatdiff['lon']*degree2km*np.cos(np.radians(lonlat2mean.lat.data[1:]))})
lonlatdiff=lonlatdiff.assign({'length':np.sqrt(lonlatdiff['x']**2+lonlatdiff['y']**2)})
lonlatdiff=lonlatdiff.assign({'costheta':lonlatdiff['x']/lonlatdiff['length']})
lonlatdiff=lonlatdiff.assign({'sintheta':lonlatdiff['y']/lonlatdiff['length']})

total_length = lonlatdiff.length.sum().data 
total_osnap_length = lonlatdiff.length[0:12].sum().data;  # exclude section across UK - just there for testing north/south

length_west = xr.concat((xr.DataArray([0],dims=("dim_0"),coords={"dim_0": [0]}),lonlatdiff.length.cumsum()),dim='dim_0')

### tracks

In [ ]:
ds=[]
for filename in filename_tracks:
    ds.append(xr.open_dataset(project_path / path_data_tracks / filename))
# ds.isel(obs=0).z.max()

#### Velocity conversions from degrees lat/lon per second to m/s

In [ ]:
for i in range(len(ds)):
    ds[i]=ds[i].assign({'uvel_ms':ds[i].uvel * degree2km * 1000.0 * np.cos(np.radians(ds[i].lat))})
    ds[i]=ds[i].assign({'vvel_ms':ds[i].vvel * degree2km * 1000.0})

#### Find initial velocities normal to the section

In [ ]:
ds_init=[]

for dsi in ds:
    ds_temp = dsi.isel(obs=0)
    ds_temp = ds_temp.assign({'section_index':xr.DataArray(np.searchsorted(lonlat.lon,ds_temp.lon)-1,dims='traj')})
    ds_temp = ds_temp.assign({'u_normal':ds_temp.vvel_ms * lonlatdiff.costheta[ds_temp.section_index].data - ds_temp.uvel_ms * lonlatdiff.sintheta[ds_temp.section_index].data})
    ds_temp = ds_temp.assign({'u_along':ds_temp.vvel_ms * lonlatdiff.sintheta[ds_temp.section_index].data + ds_temp.uvel_ms * lonlatdiff.costheta[ds_temp.section_index].data})
    ds_init.append(ds_temp)

#### Find along-section distances of initial points

In [ ]:
for i in range(len(ds_init)):
    ds_init[i] = ds_init[i].assign({'x':xr.DataArray(length_west[ds_init[i].section_index] + lonlatdiff.length[ds_init[i].section_index]*
                              (ds_init[i].lon - lonlat.lon[ds_init[i].section_index])/lonlatdiff.lon[ds_init[i].section_index],dims='traj')})

## Plot section

In [ ]:
sns.set(style="whitegrid")
central_lon, central_lat = -30, 55
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.Orthographic(central_lon, central_lat)})
extent = [-60, 0, 40, 70]
ax.set_extent(extent)
ax.gridlines()
ax.coastlines(resolution='50m')

lonlat.plot.scatter(ax=ax,transform=ccrs.PlateCarree(),x='lon',y='lat')
lonlat2mean.plot.scatter(ax=ax,transform=ccrs.PlateCarree(),x='lon',y='lat');

## Have a quick look

### Release positions of the particles

In [ ]:
def plot_ts_sections(ds_1_init,ds_2_init):
    sns.set(style="darkgrid")
    fig,ax = plt.subplots(3,2,figsize = (18,12),sharex=True,sharey=True,gridspec_kw={'wspace': 0.0,'hspace': 0.1})
    
    pcm = ax[0,0].scatter(ds_1_init.x.data.flatten(),
                        ds_1_init.z.data.flatten(),
                        2,
                        ds_1_init.salt.data.flatten(),
                        vmin=34.5,vmax=35.5,
                        cmap=co.cm.haline)
    ax[0,0].set_ylim(0,4000)
    ax[0,0].invert_yaxis()
    ax[0,0].set_ylabel('depth [m]')
    ax[0,0].set_title(ds_1_init.time[0].data.astype(str))

    fig.colorbar(pcm,ax=ax[0,0],label = "Salinity [PSU]",extend='both')

    pcm = ax[1,0].scatter(ds_2_init.x.data.flatten(),
                        ds_2_init.z.data.flatten(),
                        2,
                        ds_2_init.salt.data.flatten(),
                        vmin = 34.5, vmax = 35.5,
                        cmap=co.cm.haline)
    ax[1,0].set_ylim(0,4000)
    ax[1,0].invert_yaxis()
    ax[1,0].set_ylabel('depth [m]')
    ax[1,0].set_title(ds_2_init.time[0].data.astype(str))
    fig.colorbar(pcm,ax=ax[1,0],label = "Salinity [PSU]",extend='both')

    pcm = ax[2,0].scatter(ds_1_init.x.data.flatten(),
                        ds_1_init.z.data.flatten(),
                        2,
                        (ds_2_init-ds_1_init).salt.data.flatten(),
                        vmin = -0.3, vmax = 0.3,
                        cmap=co.cm.delta)
    ax[2,0].set_ylim(0,4000)
    ax[2,0].invert_yaxis()
    ax[2,0].set_xlabel('distance [km]')
    ax[2,0].set_ylabel('depth [m]')
    ax[2,0].set_title('difference')
    fig.colorbar(pcm,ax=ax[2,0],label = "Salinity difference [PSU]",extend='both')

    pcm = ax[0,1].scatter(ds_1_init.x.data.flatten(),
                        ds_1_init.z.data.flatten(),
                        2,
                        ds_1_init.temp.data.flatten(),
                        vmin=-1,vmax=11,
                        cmap=co.cm.thermal)
    ax[0,1].set_ylim(0,4000)
    ax[0,1].invert_yaxis()
    ax[0,1].set_title(ds_1_init.time[0].data.astype(str))
    fig.colorbar(pcm,ax=ax[0,1],label = "potential temperature [$\degree$C]",extend='both')

    pcm = ax[1,1].scatter(ds_2_init.x.data.flatten(),
                        ds_2_init.z.data.flatten(),
                        2,
                        ds_2_init.temp.data.flatten(),
                        vmin=-1,vmax=11,
                        cmap=co.cm.thermal)
    ax[1,1].set_ylim(0,4000)
    ax[1,1].invert_yaxis()
    ax[1,1].set_title(ds_2_init.time[0].data.astype(str))
    fig.colorbar(pcm,ax=ax[1,1],label = "potential temperature [$\degree$C]",extend='both')

    pcm = ax[2,1].scatter(ds_1_init.x.data.flatten(),
                        ds_1_init.z.data.flatten(),
                        2,
                        (ds_2_init-ds_1_init).temp.data.flatten(),
                        vmin = -3, vmax = 3,
                        cmap=co.cm.diff)
    ax[2,1].set_ylim(0,4000)
    ax[2,1].invert_yaxis()
    ax[2,1].set_xlabel('distance [km]')
    ax[2,1].set_title('difference')
    fig.colorbar(pcm,ax=ax[2,1],label = "temperature difference [$\degree$C]",extend='both')



## July particle releases, temperature and salinity at release. Compared with 2006 base.

### July 2007 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[1])

### July 2008 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[2])

### July 2009 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[3])

### July 2010 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[4])

### July 2011 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[5])

### July 2012 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[6])

### July 2013 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[7])

### July 2014 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[8])

### July 2015 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[9])

### July 2016 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[10])

### July 2017 vs. July 2006

In [ ]:
plot_ts_sections(ds_init[0],ds_init[11])

## January particle releases, temperature and salinity at release. Compared with 2006 base.

### January 2007 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[13])

### January 2008 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[14])

### January 2009 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[15])

### January 2010 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[16])

### January 2011 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[17])

### January 2012 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[18])

### January 2013 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[19])

### January 2014 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[20])

### January 2015 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[21])

### January 2016 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[22])

### January 2017 vs. January 2006

In [ ]:
plot_ts_sections(ds_init[12],ds_init[23])

## plot tracks by source region

In [ ]:
# range of stations from west to east, stations 0-12. Python indexing.

def conditionalplotTracksBetweenLonsCartopy(ds,condition,lonRange,depthRange,cmap=co.cm.tempo_r):
    
    dsmask = ds.where(ds.isel(obs=0).lon > lonRange[0]).where(ds.isel(obs=0).lon < lonRange[1])
    dsmask = dsmask.where(ds.isel(obs=0).z > depthRange[0]).where(ds.isel(obs=0).z < depthRange[1])
    
    dscond = dsmask.where(condition).dropna('traj', how='all')

    central_lon, central_lat = -30, 50
    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize = (16,10),subplot_kw={'projection': ccrs.Orthographic(central_lon, central_lat)})
    extent = [-60, 0, 30, 70]
    ax.set_extent(extent)
    ax.gridlines()
    ax.coastlines(resolution='50m')


    pcm = ax.scatter(
        dscond.lon.data.flatten(),
        dscond.lat.data.flatten(),
        3,
#         (dsmask.time.data.flatten()-ds.time.isel(obs=0,traj=0).data).astype('timedelta64[D]')
        mdates.date2num(dscond.time.data.flatten())
        ,cmap= cmap
        ,zorder=2
        ,transform=ccrs.PlateCarree()
    #   ,alpha=0.3
    )
    cb = fig.colorbar(pcm,ax=ax,label = "date")
    loc = mdates.MonthLocator()
    cb.ax.yaxis.set_major_locator(loc)
    cb.ax.yaxis.set_major_formatter(mdates.ConciseDateFormatter(loc))


    depth.isel(y=slice(1400, 2499), x=slice(0, 2404)).plot.contour(ax=ax,transform=ccrs.PlateCarree(),
        x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500],zorder=1
    );
#     depth.isel(y=slice(1400, 2499), x=slice(0, 2404)).plot.contour(ax=ax,transform=ccrs.PlateCarree(),
#         x="nav_lon", y="nav_lat", colors = 'k', levels = [1],zorder=3
#     );

    ax.scatter(
        ds.lon.isel(obs=0).data.flatten(),
        ds.lat.isel(obs=0).data.flatten(),2,zorder=4,
        transform=ccrs.PlateCarree()
    )
    ax.scatter(
        dsmask.lon.isel(obs=0).data.flatten(),
        dsmask.lat.isel(obs=0).data.flatten(),2,zorder=5,
        transform=ccrs.PlateCarree()
    )

    return


In [ ]:
def apply_left_of_line(ds, lon_1, lon_2, lat_1, lat_2):
    '''Apply an area crossing criterion.
    
    Larvae in ds selected while they are in a selected area.
    '''
    # particles are selected if they pass through given area.
    position =  ((lon_2 -lon_1) * (ds.lat - lat_1) - 
                     (ds.lon - lon_1) * (lat_2 - lat_1))
                        
    return position >= 0.0, position < 0

#### OSNAP line cross longitude and depth range

In [ ]:
lonRange=[-30,-20]
depthRange=[0,500]

#### from Labrador sea or from Gulf Stream

In [ ]:
ds_lab_sea_in=[]
ds_lab_sea_notin=[]
ds_60w_in=[]
ds_60w_notin=[]

for i in range(len(ds)):
    ds_in, ds_notin = apply_left_of_line(ds[i],-75,-40,40,65)
    ds_lab_sea_in.append(ds_in)
    ds_lab_sea_notin.append(ds_notin)
    ds_in, ds_notin = apply_left_of_line(ds[i],-60,-60,40,65)
    ds_60w_in.append(ds_in)
    ds_60w_notin.append(ds_notin)

##### from Labrador Sea

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[0],ds_lab_sea_in[0].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[1],ds_lab_sea_in[1].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[2],ds_lab_sea_in[2].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[3],ds_lab_sea_in[3].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[4],ds_lab_sea_in[4].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[5],ds_lab_sea_in[5].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[6],ds_lab_sea_in[6].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[7],ds_lab_sea_in[7].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[8],ds_lab_sea_in[8].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[9],ds_lab_sea_in[9].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[10],ds_lab_sea_in[10].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[11],ds_lab_sea_in[11].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[12],ds_lab_sea_in[12].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[13],ds_lab_sea_in[13].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[14],ds_lab_sea_in[14].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[15],ds_lab_sea_in[15].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[16],ds_lab_sea_in[16].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[17],ds_lab_sea_in[17].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[18],ds_lab_sea_in[18].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[19],ds_lab_sea_in[19].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[20],ds_lab_sea_in[20].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[21],ds_lab_sea_in[21].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[22],ds_lab_sea_in[22].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:

conditionalplotTracksBetweenLonsCartopy(ds[23],ds_lab_sea_in[23].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

##### from Gulf Stream

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[0],(ds_lab_sea_in[0].max("obs")==False)*ds_60w_in[0].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[1],(ds_lab_sea_in[1].max("obs")==False)*ds_60w_in[1].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[2],(ds_lab_sea_in[2].max("obs")==False)*ds_60w_in[2].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[3],(ds_lab_sea_in[3].max("obs")==False)*ds_60w_in[3].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[4],(ds_lab_sea_in[4].max("obs")==False)*ds_60w_in[4].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[5],(ds_lab_sea_in[5].max("obs")==False)*ds_60w_in[5].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[6],(ds_lab_sea_in[6].max("obs")==False)*ds_60w_in[6].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[7],(ds_lab_sea_in[7].max("obs")==False)*ds_60w_in[7].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[8],(ds_lab_sea_in[8].max("obs")==False)*ds_60w_in[8].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[9],(ds_lab_sea_in[9].max("obs")==False)*ds_60w_in[9].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[10],(ds_lab_sea_in[10].max("obs")==False)*ds_60w_in[10].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[11],(ds_lab_sea_in[11].max("obs")==False)*ds_60w_in[11].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[12],(ds_lab_sea_in[12].max("obs")==False)*ds_60w_in[12].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[13],(ds_lab_sea_in[13].max("obs")==False)*ds_60w_in[13].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[14],(ds_lab_sea_in[14].max("obs")==False)*ds_60w_in[14].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[15],(ds_lab_sea_in[15].max("obs")==False)*ds_60w_in[15].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[16],(ds_lab_sea_in[16].max("obs")==False)*ds_60w_in[16].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[17],(ds_lab_sea_in[17].max("obs")==False)*ds_60w_in[17].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[18],(ds_lab_sea_in[18].max("obs")==False)*ds_60w_in[18].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[19],(ds_lab_sea_in[19].max("obs")==False)*ds_60w_in[19].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[20],(ds_lab_sea_in[20].max("obs")==False)*ds_60w_in[20].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[21],(ds_lab_sea_in[21].max("obs")==False)*ds_60w_in[21].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[22],(ds_lab_sea_in[22].max("obs")==False)*ds_60w_in[22].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[23],(ds_lab_sea_in[23].max("obs")==False)*ds_60w_in[23].max("obs"),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

##### the rest

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[0],(ds_lab_sea_in[0].max("obs")==False)*(ds_60w_in[0].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[1],(ds_lab_sea_in[1].max("obs")==False)*(ds_60w_in[1].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[2],(ds_lab_sea_in[2].max("obs")==False)*(ds_60w_in[2].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[3],(ds_lab_sea_in[3].max("obs")==False)*(ds_60w_in[3].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[4],(ds_lab_sea_in[4].max("obs")==False)*(ds_60w_in[4].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[5],(ds_lab_sea_in[5].max("obs")==False)*(ds_60w_in[5].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[6],(ds_lab_sea_in[6].max("obs")==False)*(ds_60w_in[6].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[7],(ds_lab_sea_in[7].max("obs")==False)*(ds_60w_in[7].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[8],(ds_lab_sea_in[8].max("obs")==False)*(ds_60w_in[8].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[9],(ds_lab_sea_in[9].max("obs")==False)*(ds_60w_in[9].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[10],(ds_lab_sea_in[10].max("obs")==False)*(ds_60w_in[10].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[11],(ds_lab_sea_in[11].max("obs")==False)*(ds_60w_in[11].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[12],(ds_lab_sea_in[12].max("obs")==False)*(ds_60w_in[12].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[13],(ds_lab_sea_in[13].max("obs")==False)*(ds_60w_in[13].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[14],(ds_lab_sea_in[14].max("obs")==False)*(ds_60w_in[14].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[15],(ds_lab_sea_in[15].max("obs")==False)*(ds_60w_in[15].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[16],(ds_lab_sea_in[16].max("obs")==False)*(ds_60w_in[16].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[17],(ds_lab_sea_in[17].max("obs")==False)*(ds_60w_in[17].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[18],(ds_lab_sea_in[18].max("obs")==False)*(ds_60w_in[18].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[19],(ds_lab_sea_in[19].max("obs")==False)*(ds_60w_in[19].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[20],(ds_lab_sea_in[20].max("obs")==False)*(ds_60w_in[20].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[21],(ds_lab_sea_in[21].max("obs")==False)*(ds_60w_in[21].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[22],(ds_lab_sea_in[22].max("obs")==False)*(ds_60w_in[22].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
conditionalplotTracksBetweenLonsCartopy(ds[23],(ds_lab_sea_in[23].max("obs")==False)*(ds_60w_in[23].max("obs")==False),lonRange=lonRange,depthRange=depthRange,cmap=co.cm.tempo_r)

In [ ]:
# conda list